#### Mawerere James B20902 M23M19/003

##### Mosquito_Dataset.xlsx  Mosquito_Dataset.xlsxDownload Mosquito_Dataset.xlsx
##### This dataset contains survey data collected by scientists working on the mosquito pathogen carriers that cause infections amongst humans  in three villages in Uganda. Further information on the variables collected can be found under the sheet "Variables_Descriptors" within the excel book.


>> Instructions: 
A new infection case has been noted in Lwengo village. 


>> Qn 1. Determine the mosquito species (species.mol) that has led to this infection. 

### Import the libraries

In [1]:
## Import all packages and specify that graphs be placed inline.
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import seaborn as sns
import numpy as np

%matplotlib inline

#### Read the data

In [45]:
data_msqt = pd.read_excel("Mosquito_Dataset.xlsx")
data_msqt.head()

,village,collection,origin,fed,parity,species.mol,oocyst,spz,infection,infection1,choice,RS_tech,spz_tech
0,kitamilo,spray,CA,yes,NaN,g,3,0,oocyst,oocyst,other,pcr,qpcr
1,kitamilo,spray,MH,yes,NaN,g,0,1,spz,spz,H,pcr,qpcr
2,kitamilo,spray,MH,yes,NaN,g,0,1,spz,spz,H,pcr,qpcr
3,kitamilo,spray,MI,yes,NaN,g,0,1,spz,spz,H,pcr,qpcr
4,kitamilo,spray,MI,yes,NaN,c,0,1,spz,spz,H,pcr,qpcr


In [25]:
# Summarize the data statistically
data_msqt.describe()

,oocyst,spz
count,2912.000000,2912.000000
mean,0.335165,0.080014
std,3.303126,0.271361
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,78.000000,1.000000


#### Cleaning the datafram

In [5]:
#Replace '?' with NaN
data_msqt.replace('?', pd.NA, inplace=True)
data_msqt

,village,collection,origin,fed,parity,species.mol,oocyst,spz,infection,infection1,choice,RS_tech,spz_tech
0,kitamilo,spray,CA,yes,NaN,g,3,0,oocyst,oocyst,other,pcr,qpcr
1,kitamilo,spray,MH,yes,NaN,g,0,1,spz,spz,H,pcr,qpcr
2,kitamilo,spray,MH,yes,NaN,g,0,1,spz,spz,H,pcr,qpcr
3,kitamilo,spray,MI,yes,NaN,g,0,1,spz,spz,H,pcr,qpcr
4,kitamilo,spray,MI,yes,NaN,c,0,1,spz,spz,H,pcr,qpcr
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,buvuma,spray,MH,yes,NaN,NaN,0,0,uninfected,uninfected,other,ELISA,ELISA
2908,buvuma,spray,MH,yes,NaN,NaN,0,0,uninfected,uninfected,other,ELISA,ELISA
2909,buvuma,spray,MH,yes,NaN,NaN,0,0,uninfected,uninfected,other,ELISA,ELISA
2910,buvuma,spray,MH,yes,NaN,NaN,0,0,uninfected,uninfected,other,ELISA,ELISA


#### View the columns

In [33]:
data_msqt.columns

Index(['village', 'collection', 'origin', 'fed', 'parity', 'species.mol ',
       'oocyst', 'spz', 'infection', 'infection1', 'choice', 'RS_tech',
       'spz_tech'],
      dtype='object')

#### Check the data types

In [27]:
data_msqt.dtypes

village         object
collection      object
origin          object
fed             object
parity          object
species.mol     object
oocyst           int64
spz              int64
infection       object
infection1      object
choice          object
RS_tech         object
spz_tech        object
dtype: object

#### Replace Missing Data

In [10]:
def clean_data(data_msqt):
  for col in data_msqt.select_dtypes(include=['object']).columns:
     data_msqt = data_msqt.fillna({col: data_msqt[col].mode()[0]})
  return data_msqt

In [11]:
data_clean = clean_data(data_msqt)
data_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2912 entries, 0 to 2911
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   village       2912 non-null   object
 1   collection    2912 non-null   object
 2   origin        2912 non-null   object
 3   fed           2912 non-null   object
 4   parity        2912 non-null   object
 5   species.mol   2912 non-null   object
 6   oocyst        2912 non-null   int64 
 7   spz           2912 non-null   int64 
 8   infection     2912 non-null   object
 9   infection1    2912 non-null   object
 10  choice        2912 non-null   object
 11  RS_tech       2912 non-null   object
 12  spz_tech      2912 non-null   object
dtypes: int64(2), object(11)
memory usage: 295.9+ KB


#### Convert Categoric to Numeric

In [12]:
columns_list = data_clean.select_dtypes(include=['object']).columns

for col in columns_list:
    data_clean[col] = pd.to_numeric(data_clean[col], errors='coerce')

data_clean.dtypes

village         float64
collection      float64
origin          float64
fed             float64
parity          float64
species.mol     float64
oocyst            int64
spz               int64
infection       float64
infection1      float64
choice          float64
RS_tech         float64
spz_tech        float64
dtype: object

#### Check Missing Values

In [13]:
missing_info = data_clean.isnull().sum()
missing_info

village         2912
collection      2912
origin          2912
fed             2912
parity          2912
species.mol     2912
oocyst             0
spz                0
infection       2912
infection1      2912
choice          2912
RS_tech         2912
spz_tech        2912
dtype: int64

In [14]:
clean_data

<function __main__.clean_data(data_msqt)>

In [15]:
data_clean

,village,collection,origin,fed,parity,species.mol,oocyst,spz,infection,infection1,choice,RS_tech,spz_tech
0,NaN,NaN,NaN,NaN,NaN,NaN,3,0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2907,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2908,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2909,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2910,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN


In [56]:
data_msqt.columns = data_msqt.columns.str.strip()

In [57]:
data_msqt.dropna(inplace=True)

In [58]:
## Separating features x and y
X = data_msqt.drop(columns = ["species.mol"])
y = data_msqt["species.mol"]

In [72]:
## Training
X_train, X_test, y_train, y_train = train_test_split(X, y, test_size=0.2, random_state = 45)

In [73]:
# Separated nums from cats
numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X_train.select_dtypes(exclude=np.number).columns.tolist()

In [74]:
# clean the numerical first
from sklearn.impute import SimpleImputer

numerical_imp = SimpleImputer(strategy='mean')
X_train_num_imp = numerical_imp.fit_transform(X_train[numerical_cols])
X_test_num_imp = numerical_imp.fit_transform(X_test[numerical_cols])

In [75]:
# clean the categrorical data
categorical_imp = SimpleImputer(strategy='most_frequent')
X_train_cat_imp = categorical_imp.fit_transform(X_train[categorical_cols])
X_test_cat_imp = categorical_imp.fit_transform(X_test[categorical_cols])

In [76]:
# handling encoding  - turning cat data into numerical
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
X_train_cat_encode = encoder.fit_transform(X_train_cat_imp)
X_test_cat_encode = encoder.fit_transform(X_test_cat_imp)

In [77]:
# joined / concatenate the numericals with the encoded
X_train_processed = np.concatenate([X_train_num_imp, X_train_cat_encode.toarray()], axis=1)
X_test_processed = np.concatenate([X_test_num_imp, X_test_cat_encode.toarray()], axis=1)

### Run the model

In [78]:
#creating
model = LogisticRegression()
model.fit(X_train_processed, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [162, 41]

In [71]:
X_train.head()
X_test.head()

,village,collection,origin,fed,parity,oocyst,spz,infection,infection1,choice,RS_tech,spz_tech
180,kitamilo,spray,MH,yes,P,0,0,uninfected,uninfected,H,pcr,qpcr
11,kitamilo,spray,MH,yes,N,0,0,uninfected,uninfected,H,pcr,qpcr
193,kitamilo,spray,CA,yes,P,0,0,uninfected,uninfected,H,pcr,qpcr
125,kitamilo,spray,MH,yes,P,11,1,spz,oocspz,H,pcr,qpcr
634,bukomansimbi,obet,PA,no,P,0,0,uninfected,uninfected,other,ELISA,qpcr


In [26]:
# No. 1 reading the data, specify the sheet 
# No. 2 Preprocessing - preparing the data
# No. 3 The missing values (puting in the missing values mean,)
# No. 4 specify the training

# No. 5 Putting the models


# No. 6 ()

#### Question 2: This dataset includes measurements of cortisol, testosterone, and progesterone hormones in wolf hair samples collected from hunters in the tundra-taiga and northern boreal forests of Canada.  Additional samples were collected from wolves killed as part of a control program in the boreal forest (Little Smoky area). Detailed descriptions of the variables can be found under the sheet "Data_Descriptors" within the excel book.

#### Qn 2a. Predict the level of testosterone hormone among the wolves in the sheet labelled

#### Import Libraries

In [48]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, StandardScaler
import statsmodels.api as sm
import seaborn as sns
import numpy as np